In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import sys
import os
import pandas as pd

b:\Projects\portfolio_management\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
path = kagglehub.dataset_download("khalilvandian/portfolio-management")

# List all files and directories in the downloaded dataset path
files = os.listdir(path)
print("Files in dataset directory:", files)

100%|██████████| 651k/651k [00:00<00:00, 3.72MB/s]

Extracting files...
Files in dataset directory: ['AVGO.csv', 'AXP.csv', 'BAC.csv', 'CB.csv', 'CMG.csv', 'EA.csv', 'EBAY.csv', 'GRMN.csv', 'IBM.csv', 'IT.csv', 'LEG.csv', 'MHK.csv', 'MS.csv', 'ORLY.csv', 'XL.csv']


In [20]:
avgo = pd.read_csv(os.path.join(path, files[0]))
avgo = avgo.set_index(pd.to_datetime(avgo["Date"])).drop(columns=["Date"])
avgo.sort_index(inplace=True)
avgo["High"][0]

C:\Users\BlackDEATH\AppData\Local\Temp\ipykernel_14640\2928121268.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  avgo["High"][0]


'$14.349'

In [ ]:
# read all files into their respective data frames

'C:\\Users\\BlackDEATH\\.cache\\kagglehub\\datasets\\khalilvandian\\portfolio-management\\versions\\1'

In [ ]:
import backtrader as bt
%matplotlib inline

class BuyAndHoldCustom(bt.Strategy):

    # Add a parameters class to accept allocations
    params = (('allocations', {}),)

    def __init__(self):
        self.bought = {}
        # Access the allocations from the parameters
        self.allocations = self.p.allocations

    def next(self):
        for data in self.datas:
            name = data._name
            if not self.getposition(data).size and name not in self.bought:
                cash = self.broker.get_cash()
                allocation = self.params.allocations[name]
                size = int(allocation / data.close[0])
                self.buy(data=data, size=size)
                self.bought[name] = True

    def stop(self):
        # Called at the end of the backtest
        for data in self.datas:
            position = self.getposition(data)
            if position.size > 0:
                self.sell(data=data, size=position.size)
                print(f"SELL at END: {data._name} @ {data.close[0]:.2f}, Size: {position.size}")

def run_custom_backtest(dataframes_dict, allocations_dict, start_date, end_date):
    cerebro = bt.Cerebro()
    cerebro.broker.set_cash(sum(allocations_dict.values()))

    filtered_dataframes = {}
    for stock_name, df in dataframes_dict.items():
        df = df.copy()
        df['Date'] = pd.to_datetime(df['date'])
        df.set_index('Date', inplace=True)
        df = df.loc[start_date:end_date]
        df.columns = df.columns.str.capitalize()  # Ensure 'Close' column exists
        filtered_dataframes[stock_name] = df

        feed = bt.feeds.PandasData(dataname=df)
        cerebro.adddata(feed, name=stock_name)

    # Pass allocations into strategy as parameter
    cerebro.addstrategy(BuyAndHoldCustom, allocations=allocations_dict)

    start_value = cerebro.broker.getvalue()
    cerebro.run()
    end_value = cerebro.broker.getvalue()

    return {
        'start_value': start_value,
        'end_value': end_value,
        'total_gain': end_value - start_value,
        'percent_gain': (end_value - start_value) / start_value * 100,
        'cerebro': cerebro
    }


In [ ]:
position_size_dict = {}
for index, row in investment_strategy.iterrows():
    position_size_dict[row['Symbol']] = row['Position_Size']

# create company dicts
dataframes_dict = {}
for index, row in selected_companies.iterrows():
  ticker = row["Symbol"]

  ticker_df = sampled_stock_data[sampled_stock_data["Ticker"] == ticker]
  dataframes_dict[ticker] = ticker_df



result = run_custom_backtest(
    dataframes_dict=dataframes_dict,
    allocations_dict=position_size_dict,
    start_date='2017-08-01',
    end_date='2018-02-07'
)

print("Initial Value:", result['start_value'])
print("Final Value:", result['end_value'])
print("Total Gain:", result['total_gain'])
print("Percent Gain:", result['percent_gain'], "%")


SELL at END: CMG @ 272.21, Size: 19
SELL at END: MHK @ 267.66, Size: 26
SELL at END: GRMN @ 63.08, Size: 134
SELL at END: LEG @ 45.78, Size: 138
SELL at END: ORLY @ 252.08, Size: 32
SELL at END: BAC @ 31.25, Size: 268
SELL at END: MS @ 54.55, Size: 141
SELL at END: XL @ 42.00, Size: 150
SELL at END: CB @ 147.52, Size: 44
SELL at END: AXP @ 93.61, Size: 77
SELL at END: IBM @ 153.85, Size: 46
SELL at END: EBAY @ 42.33, Size: 186
SELL at END: EA @ 123.05, Size: 57
SELL at END: AVGO @ 237.38, Size: 26
SELL at END: IT @ 119.45, Size: 52
Initial Value: 100500.0
Final Value: 106374.05999999998
Total Gain: 5874.059999999983
Percent Gain: 5.844835820895505 %


In [ ]:
import matplotlib
matplotlib.use('Agg')  # Use a non-interactive backend for saving

cerebro = result["cerebro"]
# Generate plot
figs = cerebro.plot(style='candlestick')  # You can choose style: line, candlestick, ohlc

# Set large size and save the first figure
fig = figs[0][0]  # First subplot in first figure
fig.set_size_inches(24, 50)  # Width, Height in inches
fig.savefig("large_backtest_plot.png", dpi=300)  # Save with high resolution